# **Resolución de un problema mediante búsqueda heurística A**

![texto alternativo](https://marketing4ecommerce.net/wp-content/uploads/2017/04/robots-log%C3%ADsticos-de-Amazon-imagen-caribbean-digital.png)


## Integrantes del grupo

Grupo 18 ‐ Equipo 3

- Adrián Bañuls Arias
- Francisco José Cobo Celdrán
- Carles Serra Vendrell
- Miguel Jiménez Gomis
- Miriam Pardo Cuesta

# Asunciones

Para la resolución del problema, se ha asumido que el robot conoce el mapa y la ubicación de las mesas, además de la posición objetivo de cada mesa. Para calcular el coste estimado para alcanzar el nodo objetivo se hace uso de la distancia Manhattan. Por otro lado, el coste real se ha asumido el mismo para todos los nodos.

Se ha asumido también que las mesas no son obstáculos para el robot, tanto para cuando el robot va sin cargar como para cuando va cargado.

Por último, se ha asumido que el orden en el que se recogen las mesas sigue la numeración de las mesas. Es decir, el robot primero lleva M1 a su meta, luego M2 y finalmente M3.

![texto alternativo](https://drive.google.com/uc?export=view&id=1Mzml6909YsiwqCB77-xeLYRoVt93uLKg)

# Desarrollo de la actividad

En primer lugar, se han definido variables globales al dominio del problema como son el tamaño del mapa, el propio mapa, y las direcciones posibles (Arriba, Abajo, Izquierda, Derecha). También, para facilitar el desarrollo, las listas cerrada y abierta del algoritmo A* se han definido como variables globales. Si bien para una aplicación de uso comercial estas listas se tendrían que englobar dentro de un objeto, para simplificar el desarrollo hemos decidido mantenerlas así.

Se ha definido la clase *nodos* en la cual se reflejan los posibles puntos por los que el agente puede moverse así como información relevante para el problema (distancia ya recorrida y prioridad del nodo). A continuación, se ha definido la función *pathFind* en la cual se ha codificado el algoritmo A* , que tomando uso de una posición inicial, una posición final y las listas globales declaradas, calcula la ruta entre dichas posiciones. Por último, se ha definido la función *main*, en la que se describe el dominio del problema concreto a resolver y se codifica una sencilla interfaz para poder interpretar los resultados del algoritmo diseñado.

In [1]:
import queue as Q

#Se definen las variables globales del problema
n = 4 #Tamaño horizontal del entorno del agente (Mapa) CAMBIAR SI SE PRECISA DE OTRAS DIMENSIONES
m = 4 #Tamaño vertical del entorno del agente (Mapa)   CAMBIAR SI SE PRECISA DE OTRAS DIMENSIONES
mapa = [] #Mapa global del problema
lista_cerrada_nodos = [] # Listado cerrado, es decir, valores de los nodos que se han intentado visitar
lista_abierta_nodos = [] # Listado de nodos abiertos, es decir nodos sin visitar
dircc_padres_mapa = [] # Mapa de las direcciones para saber donde se han realizado los movimientos
dir = 4 #num direcciones posibles, las direccones posibles se reflejan como pares del mismo inidice de las listas dx y dy
dx = [1,0,-1,0] #Direcciones posibles en el eje x
dy = [0,1,0,-1] #Direcciones posibles en el eje x

#Se rellenan los listados en blanco
for i in range(n):
  mapa.append([0] * m)
  lista_cerrada_nodos.append([0] * m)
  lista_abierta_nodos.append([0] * m)
  dircc_padres_mapa.append([0] * m)

class nodos(object):
    '''En esta clase se define el concepto de nodo, que son los posibles puntos por donde el agente puede moverse
    con información relevante del problema.
    '''
    def __init__(self, xp, yp, d, p):
      ''' Constructor de la clase nodo
      xp Posición x actual
      yp Posiciñon y actual
      d Distancia que ya se ha recorrido hasta el nodo
      p Prioridad del nodo (Nivel + estimacion de la distancia restante)
      '''
      self.xpos = xp
      self.ypos = yp
      self.distancia = d
      self.prioridad = p

    def updateprioridad(self, xDest, yDest):
      '''Funcion que actualiza las prioridades del nodo con respecto al destino dado
      xDest Posición x del destino
      yDest Posiciñon y del destino
      '''
      self.prioridad = self.distancia + self.estimate(xDest, yDest)

    def estimate(self, xDest, yDest):
      '''Estimación de la distancia por medio de la distancia Manhattan
      '''
      xd = xDest - self.xpos
      yd = yDest - self.ypos
      return abs(xd) + abs(yd)
       
    def __gt__(self, other):
      '''Sobrecarga del operador > para determinal la prioridad de la cola
      '''
      return self.prioridad > other.prioridad

    def __eq__(self, other):
      '''Sobrecarga del operador eq para determinal la prioridad de la cola
      '''
      return self.prioridad == other.prioridad


def pathFind(xStart, yStart, xFinish, yFinish):
      '''Esta función toma las posiciones iniciales y finales y calcula el recorrido óptimo entre ellos.
      xStart  Posicion x del inicio 
      yStart  Posicion y del inicio
      xFinish Posicion x del destino 
      yFinish Posicion y del destino
      '''
      pq = [Q.PriorityQueue(),Q.PriorityQueue()] #Se crean las 2 listas de prioridades que se utilizarán
      pqi = 0 #indice que indica que lista de prioridad está en uso 

      for y in range(m): #Se realiza una limpieza del mapa tanto de la lista cerrada como de la abierta
        for x in range(n):
          lista_cerrada_nodos[x][y] = 0
          lista_abierta_nodos[x][y] = 0

      n0 = nodos(xStart,yStart,0,0) #Se crea el primer nodo en la posicion de inicio
      n0.updateprioridad(xFinish, yFinish) #Se actualiza la prioridad del primer nodo con respecto al destino
      pq[pqi].put(n0) #Se introduce el nodo en la lista de prioridades
      lista_abierta_nodos[xStart][yStart] = n0.prioridad #se actualiza la lista abierta con la prioridad del nodo

      while not (pq[pqi].empty()):
        temp = pq[pqi].get()
        n0 = nodos(temp.xpos, temp.ypos, temp.distancia, temp.prioridad) #Se saca el nodo de mayor prioridad de la lista
        x = n0.xpos
        y = n0.ypos
        lista_abierta_nodos[x][y] = 0
        lista_cerrada_nodos[x][y] = 1 #Se marca el nodo como visitado

        if (x == xFinish and y == yFinish): #Una vez se descubre el recorrido óptimo se codifica en una cadena de texto para su fácil manipulación
          path = ""
          while not (x == xStart and y == yStart):
            j = int(dircc_padres_mapa[x][y])
            temp = int((j + dir / 2) % dir)
            c =  str(temp) 
            path = c + path
            x += dx[int(j)]
            y += dy[int(j)]
          del n0
          while not (pq[pqi].empty()):
            pq[pqi].get()
          return path
          
        for i in range(dir): #Para el nodo actual se comprueban los movimientos disponibles
          xdx = x + dx[i]
          ydy = y + dy[i]
          condicion = False
          if (xdx >= n) or (ydy >= m):
            continue
          if (xdx > -1 )and ydy > -1 and (xdx < n) and (ydy < m): #Esta condición está para evitar acceso fuera de rango
            condicion = (lista_cerrada_nodos[xdx][ydy] == 1)
          else:
            False
          if not ((xdx < 0) or (xdx > (n-1)) or (ydy < 0) or (ydy > (m - 1)) or (mapa[xdx][ydy] == 1) or condicion):
            #Se generan los nodos de los movimientos posibles, se actualiza su prioridad y se actualiza las listas correspondientes
            m0 = nodos(xdx, ydy, n0.distancia, n0.prioridad) 
            m0.updateprioridad(xFinish, yFinish)
            if lista_abierta_nodos[xdx][ydy] == 0:
              lista_abierta_nodos[xdx][ydy] = m0.prioridad
              pq[pqi].put(m0)
              dircc_padres_mapa[xdx][ydy] = (i + dir / 2) % dir
            elif lista_abierta_nodos[xdx][ydy] > m0.prioridad:
              lista_abierta_nodos[xdx][ydy] = m0.prioridad
              dircc_padres_mapa[xdx][ydy] = (i + dir / 2) % dir
              while not (pq[pqi][0].xPos == xdx and pq[pqi][0].yPos):
                pq[1 - pqi].put(pq[pqi].pop())
              pq[pqi].pop()
              if (pq[pqi].qsize() > pq[1 - pqi].qsize()):
                pqi = 1 - pqi                
              while not (pq[pqi].empty()):
                pq[1 - pqi].put(pq[pqi].pop())
              pqi = 1 - pqi
              pq[pqi].put(m0)
      return "" #Si en la ejecución no se retorna la cadena con los movimientos es porque no existe camino posible  

In [2]:
def main():
  '''Esta función se trata del cuerpo del programa en el que se define el problema 
  a resolver y se muestran los resultados obtenidos de la búsqueda A*.
  '''  
  for y in range(m): #Se limpia el mapa
    for x in range(n):
      mapa[x][y] = 0
  #Lista en donde se definen los obstáculos del problema -----------------------
  lista_obstaculos = [[1,0],[1,1]] 
  for obstaculo in lista_obstaculos:
    mapa[obstaculo[0]][obstaculo[1]] = 1
	#Se define la lista de objetivos en orden de recogida-entrega  ---------------
  lista_objetivos = [[0,0],[3,3],[0,2],[2,3],[3,0],[1,3]] 
	#Se selecciona la posición inicial del agente --------------------------------
  xA = 2  
  yA = 2
  #Se ejecuta el algoritmo y se formatea la salida
  print("Ejecución del algoritmo A*")
  print("Leyenda:\n\t~ Casilla libre\n\t# Obstáculo \n\t* Ruta seguida\n\tS Inicio\n\tF Destino alcanzado")
  print( "Tamaño del mapa (X,Y): " + str(n) + "," + str(m) )
  print("\n")  
  for indice, objetivo in enumerate(lista_objetivos): #Se buscan las rutas entre los diferentes objetivos y su destino
    for i in range(n): #Se rellenan los listados con 0 para asegurar que están limpios
      mapa.append([0] * m)
      lista_cerrada_nodos.append([0] * m)
      lista_abierta_nodos.append([0] * m)
      dircc_padres_mapa.append([0] * m)
    route = pathFind(xA, yA, objetivo[0], objetivo[1]) #Ejecución del algoritmo de búsqueda
    #Se formatea la salida
    print("Posición inicial: " + str(xA) + "," + str(yA))
    print("Posición final: " + str(objetivo[0]) + "," + str(objetivo[1]))     
    if (route == "") :
      print("No se ha encontrado ruta")
      break
    if indice % 2 == 0:
        final = "--Ruta desde el origen " + str(xA) + "," + str(yA)  + " hasta el siguiente objeto en " + str(objetivo[0]) + "," + str(objetivo[1]) + ":"
        print(final)
    else:
        final = "--Ruta desde el objeto recogido en " + str(xA) + "," + str(yA)  + " hasta su destino en " + str(objetivo[0]) + "," + str(objetivo[1]) + ":"
        print(final)
    movimientos = ""
    for char in route:
      moves = {'0':'Derecha','1':'Abajo','2':'Izquierda','3':'Arriba'}
      movimientos += moves[char] + ", "
    print(movimientos , "\n")
    if (len(route) > 0): #Se traza el movimiento realizado por el agente
      x = xA
      y = yA
      mapa[x][y] = 2
      for i in range(len(route)):
        c = route[i]
        j = int(c)
        x = x + dx[j]
        y = y + dy[j]
        mapa[x][y] = 3
        for Y in range(m): #Se imprime el estado del mapa obtenido de la cadena de movimientos
          for X in range(n):
            if (mapa[X][Y] == 0):
              print("~", end = '') #Casilla libre
            elif (mapa[X][Y] == 1):
              print("#", end = '') #Obstáculo
            elif (mapa[X][Y] == 2):
              print("S", end = '') #Posición inicial
            elif (mapa[X][Y] == 3):
              print("*", end = '') #Ruta ejecutada
            elif (mapa[X][Y] == 4):
              print("F", end = '') #Posición final
          print()
        print("-" * (n*2))
      mapa[x][y] = 4

      for Y in range(m): #Se imprime el estado FINAL del mapa obtenido de la cadena de movimientos
        for X in range(n):
          if (mapa[X][Y] == 0):
            print("~", end = '') #Casilla libre
          elif (mapa[X][Y] == 1):
            print("#", end = '') #Obstáculo
          elif (mapa[X][Y] == 2):
            print("S", end = '') #Posición inicial
          elif (mapa[X][Y] == 3):
            print("*", end = '') #Ruta ejecutada
          elif (mapa[X][Y] == 4):
            print("F", end = '') #Posición final
        print()
      print("-" * (n*2))      
      xA = objetivo[0] #Se establece el Siguiente punto de inicio con la posición alcanzada del agente
      yA = objetivo[1]
      for y in range(m): #Se limpia el mapa para su siguiente ejecución
        for x in range(n):
          mapa[x][y] = 0
      for obstaculo in lista_obstaculos: #Se vuelven a insertar los obstáculos
        mapa[obstaculo[0]][obstaculo[1]] = 1

  print("\n----------FIN DE LOS MOVIMIENTOS-------------")

In [3]:
if __name__ == "__main__": 
  main()

Ejecución del algoritmo A*
Leyenda:
	~ Casilla libre
	# Obstáculo 
	* Ruta seguida
	S Inicio
	F Destino alcanzado
Tamaño del mapa (X,Y): 4,4


Posición inicial: 2,2
Posición final: 0,0
--Ruta desde el origen 2,2 hasta el siguiente objeto en 0,0:
Izquierda, Izquierda, Arriba, Arriba,  

~#~~
~#~~
~*S~
~~~~
--------
~#~~
~#~~
**S~
~~~~
--------
~#~~
*#~~
**S~
~~~~
--------
*#~~
*#~~
**S~
~~~~
--------
F#~~
*#~~
**S~
~~~~
--------
Posición inicial: 0,0
Posición final: 3,3
--Ruta desde el objeto recogido en 0,0 hasta su destino en 3,3:
Abajo, Abajo, Derecha, Derecha, Derecha, Abajo,  

S#~~
*#~~
~~~~
~~~~
--------
S#~~
*#~~
*~~~
~~~~
--------
S#~~
*#~~
**~~
~~~~
--------
S#~~
*#~~
***~
~~~~
--------
S#~~
*#~~
****
~~~~
--------
S#~~
*#~~
****
~~~*
--------
S#~~
*#~~
****
~~~F
--------
Posición inicial: 3,3
Posición final: 0,2
--Ruta desde el origen 3,3 hasta el siguiente objeto en 0,2:
Izquierda, Izquierda, Izquierda, Arriba,  

~#~~
~#~~
~~~~
~~*S
--------
~#~~
~#~~
~~~~
~**S
--------
~#~

Como se puede comprobar, los resultados han sido satisfactorios puesto que el robot ha logrado alcanzar todos los objetivos indicados en la definición del problema. En la ejecución del algoritmo se puede observar el avance del robot en cada iteración, verificando que se cumplen los objetivos propuestos.

# Dificultades encontradas

Debido a la complejidad del algoritmo, realizar una adecuada implementación se puede volver difícil. Sobre todo al existir numerosas variables que se modifican a lo largo del problema y que influyen en la decisión del agente, depurar errores se vuelve una tarea muy complicada.

Igualmente se tomaron asunciones que se decidieron de antemano con el fin de acotar el alcance de este trabajo, ya que si no se habría vuelto un desarrollo mucho más complejo.

# Referencias bibliográficas
[1] Russell, S. J., & Norvig, P. (1995). Artificial intelligence: A modern approach. Englewood Cliffs, N.J: Prentice Hall.

[2] Hart, P. E., Nilsson, N. J., & Raphael, B. (1972). Correction to" a formal basis for the heuristic determination of minimum cost paths". ACM SIGART Bulletin, (37), 28-29.
